# Task - CafeBakery Data

### 
A somewhat busy Cafe that has a bakery provides a limited number of items that are available on its Cafe Menu. It serves both walk-in and seated customers. The location is an American Town. Assume that you are Data Miner contracted to make sense of the data, identify patterns and generate some useful knowledge from the data. A dataset is provided with transactional data over a 7 month period. October 2016 to April 2017. The Cafe is open daily. Ignore the data for October 2016 and April 2017 since it only represents data for incomplete months, the rest are complete.

#### Do the following and report in a clear way.
1. Study the data and analyze it carefully. Provide a monthly summary that include frequencies, describe the timings vs the customer flow over a working day and weekend day.
2. Convert the data in a format that you can then use on WEKA (or any other ML toolkit) to enable you use the Apriori Algorithm for Association Analysis.


### 2.1. Identify the frequent itemsets by setting your own Support and Confidence values.

In [1]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

df = pd.read_csv('data/Cafe_BakeryDataset.csv')
df.head()

,Date,Time,Transaction,Item
0,10/30/2016,9:58:11,1,Bread
1,10/30/2016,10:05:34,2,Scandinavian
2,10/30/2016,10:05:34,2,Scandinavian
3,10/30/2016,10:07:57,3,Hot chocolate
4,10/30/2016,10:07:57,3,Jam


In [2]:
#Check for missing values in the full dataset
df.isna().sum()

Date           0
Time           0
Transaction    0
Item           0
dtype: int64

#### There are no missing values

#### Remove data for October 2016 and April 2017

In [3]:
df['Date'] = df['Date'].astype('str')
df = df[~df['Date'].str.contains('10/.*/2016')]
df = df[~df['Date'].str.contains('4/.*/2017')]

In [54]:
import datetime
dates = df.drop(['Time', 'Transaction', 'Item'], axis = 1)
# print ("The first day is : " +  str(dates))
dates = [dates]
dates = np.concatenate(dates).astype(str)
# dates = dates.T
# dates = [datetime.datetime(*x) for x in dates]
dates[-1]

array(['3/31/2017'], dtype='<U10')

### get weekends

In [55]:

# import numpy as np

# create_date = dates[0]
# resolve_date = dates[-1]

# create_datetime = datetime.strptime(create_date, '%m-%d-%Y')
# resolve_datetime = datetime.strptime(resolve_date, '%m-%d-%Y')
# #
# print(f"The difference in days is: {(resolve_datetime - create_datetime).days}")
# print(f"The difference in business days is: {np.busday_count(create_datetime.date(), resolve_datetime.date())}")

### Convert the Transaction column to type INT

In [4]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['Transaction']= le.fit_transform(df['Transaction'].values)

In [5]:
df.head()

,Date,Time,Transaction,Item
385,11/1/2016,7:51:20,0,Coffee
386,11/1/2016,7:51:20,0,Pastry
387,11/1/2016,8:20:50,1,Coffee
388,11/1/2016,8:20:50,1,Pastry
389,11/1/2016,8:22:28,2,Jam


In [6]:
df.tail()

,Date,Time,Transaction,Item
20097,3/31/2017,16:53:39,8844,Coke
20098,3/31/2017,16:53:39,8844,Coffee
20099,3/31/2017,16:53:39,8844,Soup
20100,3/31/2017,17:23:57,8845,Coffee
20101,3/31/2017,17:23:57,8845,Juice


In [7]:
df.dtypes

Date           object
Time           object
Transaction     int64
Item           object
dtype: object

### Consolidate the items into 1 transaction per row with each product 1 hot encoded.
This analysis requires that all the data for a transaction be included in 1 row and the items should be 1-hot encoded. 

In [8]:
basket = (df.groupby(['Date', 'Item'])['Transaction']
          .sum().unstack().reset_index().fillna(0)
          .set_index('Date'))
basket

Item,Adjustment,Afternoon with the baker,Alfajores,Argentina Night,Art Tray,Bacon,Baguette,Bakewell,Bare Popcorn,Basket,...,The BART,The Nomad,Tiffin,Toast,Truffles,Tshirt,Valentine's card,Vegan Feast,Vegan mincepie,Victorian Sponge
Date,,,,,,,,,,,,,,,,,,,,,
1/1/2017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1/10/2017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4218.0,0.0,0.0,0.0,0.0,0.0,0.0
1/11/2017,0.0,8583.0,4285.0,0.0,4283.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,8551.0,8520.0,0.0,0.0,0.0,0.0,0.0,0.0
1/12/2017,0.0,0.0,8638.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,8660.0,25960.0,0.0,0.0,0.0,0.0,4324.0,0.0
1/13/2017,0.0,0.0,13121.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4381.0,21820.0,8762.0,0.0,0.0,0.0,8765.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3/5/2017,0.0,44239.0,7370.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,7330.0,22071.0,14666.0,7362.0,0.0,0.0,0.0,0.0,0.0
3/6/2017,0.0,7424.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,29603.0,7415.0,0.0,0.0,0.0,0.0,0.0
3/7/2017,0.0,0.0,0.0,0.0,0.0,0.0,14885.0,0.0,0.0,0.0,...,0.0,0.0,22408.0,22340.0,7475.0,0.0,0.0,0.0,0.0,0.0


### One-hot encode the dataset
There are a lot of zeros in the data but we also need to make sure any positive values are converted to a 1 and anything less the 0 is set to 0.

In [9]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)
basket_sets

Item,Adjustment,Afternoon with the baker,Alfajores,Argentina Night,Art Tray,Bacon,Baguette,Bakewell,Bare Popcorn,Basket,...,The BART,The Nomad,Tiffin,Toast,Truffles,Tshirt,Valentine's card,Vegan Feast,Vegan mincepie,Victorian Sponge
Date,,,,,,,,,,,,,,,,,,,,,
1/1/2017,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1/10/2017,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1/11/2017,0,1,1,0,1,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
1/12/2017,0,0,1,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,1,0
1/13/2017,0,0,1,0,0,0,0,0,0,0,...,0,0,1,1,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3/5/2017,0,1,1,0,0,0,0,0,0,0,...,0,1,1,1,1,0,0,0,0,0
3/6/2017,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
3/7/2017,0,0,0,0,0,0,1,0,0,0,...,0,0,1,1,1,0,0,0,0,0


### 2.2 Using this, generate frequent itemsets per month, working day vs weekend. What Association Rules do you find?

In [10]:
frequent_itemsets = apriori(basket_sets, min_support=0.40, use_colnames=True)

In [13]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.describe()

,antecedent support,consequent support,support,confidence,lift,leverage,conviction
count,2.063142e+06,2.063142e+06,2.063142e+06,2.063142e+06,2.063142e+06,2.063142e+06,2.063142e+06
mean,6.855485e-01,6.855485e-01,4.727626e-01,7.204931e-01,1.053223e+00,2.218293e-02,inf
std,1.591074e-01,1.591074e-01,7.421340e-02,1.618137e-01,4.716726e-02,1.733790e-02,NaN
min,4.054054e-01,4.054054e-01,4.054054e-01,4.054054e-01,1.000000e+00,0.000000e+00,1.000000e+00
25%,5.405405e-01,5.405405e-01,4.189189e-01,5.765766e-01,1.017989e+00,8.400292e-03,1.043592e+00
50%,6.891892e-01,6.891892e-01,4.459459e-01,7.294118e-01,1.039847e+00,1.821585e-02,1.118919e+00
75%,8.175676e-01,8.175676e-01,5.000000e-01,8.584906e-01,1.076215e+00,3.255113e-02,1.292230e+00
max,1.000000e+00,1.000000e+00,9.932432e-01,1.000000e+00,1.355311e+00,1.080168e-01,inf


In [17]:
rules[ (rules['lift'] >= 1.3) &
       (rules['confidence'] >= 0.8) ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2429,(Scone),"(Toast, Cake)",0.439189,0.709459,0.405405,0.923077,1.301099,0.093818,3.777027
9550,"(Scone, Bread)","(Toast, Cake)",0.439189,0.709459,0.405405,0.923077,1.301099,0.093818,3.777027
9556,(Scone),"(Toast, Bread, Cake)",0.439189,0.709459,0.405405,0.923077,1.301099,0.093818,3.777027
18667,"(Scone, Coffee)","(Toast, Cake)",0.439189,0.709459,0.405405,0.923077,1.301099,0.093818,3.777027
18671,(Scone),"(Toast, Coffee, Cake)",0.439189,0.709459,0.405405,0.923077,1.301099,0.093818,3.777027
...,...,...,...,...,...,...,...,...,...
1613718,"(Truffles, Cake, Bread, Pastry, Tea)","(Juice, Toast, Coffee, Sandwich)",0.500000,0.628378,0.412162,0.824324,1.311828,0.097973,2.115385
1613819,"(Coffee, Truffles, Cake, Pastry)","(Toast, Juice, Bread, Sandwich, Tea)",0.500000,0.628378,0.412162,0.824324,1.311828,0.097973,2.115385
1613822,"(Pastry, Bread, Truffles, Cake)","(Toast, Juice, Coffee, Sandwich, Tea)",0.500000,0.628378,0.412162,0.824324,1.311828,0.097973,2.115385
1613826,"(Pastry, Truffles, Cake, Tea)","(Toast, Juice, Coffee, Bread, Sandwich)",0.500000,0.628378,0.412162,0.824324,1.311828,0.097973,2.115385


### 2.3 c. Is there a drift over the months?